In [1]:
import numpy as np
from scipy.io import loadmat
import tensorflow as tf
import time
import os
import matplotlib.pyplot as plt
%matplotlib inline

print("import done")

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


import done


In [ ]:
_SOURCE_FILES =  "../data/test/*.mat"
_DEST_FOLDER = "../dataset/test/"
_NUM_FILES = None # None is the total number of files

def mat2tfr(p_file):
    # getting the filename and retrieving the patient, segement and label data
    pat, seg, label = p_file.split('/')[-1].split('.')[0].split("_")
    filename = pat + "_" + seg + "_" + label + ".tfr"
    fullpathname = _DEST_FOLDER + filename
    
    if os.path.exists(fullpathname):
        print("Dataset file", fullpathname, "already exists, skipping...")
    else: 
        t = time.time()    
        print("Converting " + p_file + " ----> " + fullpathname)
        # converting mat file as numpy
        mat = loadmat(p_file)
        data = mat['dataStruct']['data'][0][0]
        # check if file is mainly zero's (100% dropout)
        if (np.count_nonzero(data) < 10) or (np.any(np.std(data, axis=0) < 0.5)):
            print("WARNING: File %s is all dropout." %p_file)
            return
        # TensorFlow Records writer
        with tf.python_io.TFRecordWriter(fullpathname) as tfrwriter:
            # Fill protobuff
            protobuf = tf.train.Example(features=tf.train.Features(feature={
                        'data' : tf.train.Feature(float_list=tf.train.FloatList(value=data.flatten().tolist())), 
                        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(label)])), 
                        'filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename])), 
                    }))
            write = tfrwriter.write(protobuf.SerializeToString())
        elapsed = time.time() - t
        print("elapsed: %.3fs"%elapsed)
        
        
def dataset(folder, num_files=None):
    # get files
    filenames = tf.gfile.Glob(folder)
    # randomize read files
    np.random.shuffle(filenames)
    # truncate reading
    if num_files is not None:
        filenames = filenames[:num_files]
    print("Converting #%d files."%len(filenames))
    
    for files in filenames:
        mat2tfr(files)       


dataset(_SOURCE_FILES, _NUM_FILES)

print('finished')        

Converting #1908 files.
Converting ../data/test/new_3_74.mat ----> ../dataset/test/new_3_74.tfr
elapsed: 3.048s
Converting ../data/test/new_3_340.mat ----> ../dataset/test/new_3_340.tfr
elapsed: 2.807s
Converting ../data/test/new_2_122.mat ----> ../dataset/test/new_2_122.tfr
elapsed: 2.776s
Converting ../data/test/new_3_517.mat ----> ../dataset/test/new_3_517.tfr
elapsed: 2.764s
Converting ../data/test/new_3_458.mat ----> ../dataset/test/new_3_458.tfr
elapsed: 2.754s
Converting ../data/test/new_2_945.mat ----> ../dataset/test/new_2_945.tfr
elapsed: 2.797s
Converting ../data/test/new_2_367.mat ----> ../dataset/test/new_2_367.tfr
elapsed: 2.830s
Converting ../data/test/new_2_958.mat ----> ../dataset/test/new_2_958.tfr
elapsed: 2.777s
Converting ../data/test/new_2_829.mat ----> ../dataset/test/new_2_829.tfr
elapsed: 2.807s
Converting ../data/test/new_2_825.mat ----> ../dataset/test/new_2_825.tfr
elapsed: 2.807s
Converting ../data/test/new_2_192.mat ----> ../dataset/test/new_2_192.tfr
elap

In [ ]:
def plot_eeg(data):
    plt.figure(figsize=(10,20))
    for i in range(0,16):
        plt.subplot(8,2,i+1)
        plt.plot(data[:,i])
    
    #plt.savefig('foo.pdf', bbox_inches='tight')